In [4]:
def load_road_network(filename):
    intersections = {}
    roads = {}

    with open(filename, 'r') as file:
        lines = file.readlines()

        i = 0
        while i < len(lines):
            line = lines[i].strip()

            if line.startswith('#Intersection:'):
                intersection_id = int(line.split(':')[1])
                signals = []

                i += 1
                while i < len(lines) and lines[i].strip().startswith('('):
                    signal_pairs = lines[i].strip().split(';')
                    signal = [(int(pair[1]), int(pair[3])) for pair in signal_pairs]
                    signals.append(signal)
                    i += 1

                intersections[intersection_id] = signals

            elif line == '#Roads':
                i += 1
                while i < len(lines) and lines[i].strip() != '':
                    road_info = lines[i].strip().split(':')
                    road = tuple(map(int, road_info[0][1:-1].split(',')))
                    cost = int(road_info[1])
                    roads[road] = cost
                    i += 1

            else:
                i += 1

    return intersections, roads


In [7]:
filename = "/Users/hridoy/Desktop/Fiverr DS code/dummy_road_network.txt"
intersections, roads = load_road_network(filename)
print(intersections)
print(roads)


{0: [], 1: [], 2: [], 3: []}
{(0, 1): 1, (1, 0): 1, (0, 2): 2, (2, 0): 2, (0, 3): 1, (3, 0): 1, (1, 3): 2, (3, 1): 2, (2, 3): 1, (3, 2): 1, (2, 4): 1, (4, 2): 1}


In [9]:
def path_cost(path, intersections, road_times):
    total_cost = 0

    for i in range(len(path) - 1):
        start_node = path[i]
        end_node = path[i+1]

        if start_node in intersections and end_node in intersections:
            signals = intersections[start_node]
            road = (start_node, end_node)

            # Check if the road exists in the road_times dictionary
            if road not in road_times:
                return None

            cost = road_times[road]

            # Check if there are traffic signals at the start_node
            if signals:
                # Check if there is a signal that allows the road traversal
                if road in signals[total_cost % len(signals)]:
                    total_cost += cost
                else:
                    return None
            else:
                total_cost += cost

        else:
            return None

    return total_cost


In [36]:
simple_intersections = {0: [[], [(1, 2), (2, 1)]]}
simple_roads = {(0, 1): 1, (1, 0): 1, (0, 2): 1, (2, 0): 1}

path = [1, 0, 2]
cost = path_cost(path, simple_intersections, simple_roads)
print(cost)


None


In [16]:
def intersection_step(intersections, road_times, intersection_id, cars_at_intersection, timestep):
    intersection_signals = intersections[intersection_id]
    signals_count = len(intersection_signals)

    cars_to_move = []

    for car_id, path, arrival_time in cars_at_intersection:
        if len(path) > signals_count:
            signal = intersection_signals[len(path) - signals_count - 1]
            if arrival_time <= timestep and signal in path[-signals_count:]:
                cars_to_move.append(car_id)

    cars_to_move.sort()
    return cars_to_move


In [18]:
simple_intersections = {0: [[], [(1, 2), (2, 1)]]}
simple_roads = {(0, 1): 1, (1, 0): 1, (0, 2): 1, (2, 0): 1}
cars_at_intersection = [(0, [1, 0, 2], 1), (1, [2, 0, 1], 1)]

cars_to_move = intersection_step(simple_intersections, simple_roads, 0, cars_at_intersection, 1)
print(cars_to_move)


[]


In [32]:
def simulate(intersections, road_times, cars_to_add):
    actions = []
    cars_in_network = []

    timestep = 0

    while cars_to_add or cars_in_network:
        # Add new cars to the network
        while cars_to_add and cars_to_add[0][2] == timestep:
            car_id, path, _ = cars_to_add.pop(0)
            cars_in_network.append((car_id, path, 0))

        # Process intersection steps
        for intersection_id in intersections.keys():
            cars_at_intersection = [(car_id, path, step) for car_id, path, step in cars_in_network if intersection_id in path and step < len(path) - 1]
            cars_to_move = intersection_step(intersections, road_times, intersection_id, cars_at_intersection, timestep)

            for car_id in cars_to_move:
                car_index = next(index for index, (id, _, _) in enumerate(cars_in_network) if id == car_id)
                car = cars_in_network.pop(car_index)
                source_node = car[1][car[2]]
                destination_node = car[1][car[2] + 1]
                actions.append(f"drive({timestep}, {car_id}, {source_node}, {destination_node})")
                car = (car[0], car[1], car[2] + 1)
                cars_in_network.append(car)

        # Process driving and arrival
        driving_cars = []
        for index, car in enumerate(cars_in_network):
            source_node = car[1][car[2]]
            destination_node = car[1][car[2] + 1]
            travel_time = road_times[(source_node, destination_node)]
            if car[2] == len(car[1]) - 2:
                actions.append(f"arrive({timestep+travel_time}, {car[0]}, {destination_node})")
            else:
                actions.append(f"drive({timestep}, {car[0]}, {source_node}, {destination_node})")
                driving_cars.append((car[0], car[1], car[2]+1))

        cars_in_network = driving_cars

        timestep += 1

    return actions


In [33]:
simple_intersections = {0: [[], [(1, 2), (2, 1)]]}
simple_roads = {(0, 1): 1, (1, 0): 1, (0, 2): 1, (2, 0): 1}
cars_to_add = [(0, [1, 0, 2], 0)]

actions = simulate(simple_intersections, simple_roads, cars_to_add)
print(actions)


['drive(0, 0, 1, 0)', 'arrive(2, 0, 2)']


In [34]:
def test_simulate(input_args, expected_return_value):
    result = simulate(*input_args)
    assert result == expected_return_value, f"Expected {expected_return_value}, but got {result}"


In [31]:
# Test Case 1
intersections = {
    0: [[], [(1, 2), (2, 1)]],
    1: [[(2, 0)], []]
}
road_times = {(0, 1): 1, (1, 0): 1, (0, 2): 1, (2, 0): 1}
cars_to_add = [
    (0, [1, 0, 2], 0),
    (1, [2, 0, 1], 1)
]
expected_actions = [
    'drive(0, 0, 1, 0)',
    'drive(1, 0, 0, 2)',
    'arrive(2, 0, 2)',
    'drive(2, 1, 2, 0)',
    'drive(3, 1, 0, 1)',
    'arrive(4, 1, 1)'
]
test_simulate((intersections, road_times, cars_to_add), expected_actions)


AssertionError: Expected ['drive(0, 0, 1, 0)', 'drive(1, 0, 0, 2)', 'arrive(2, 0, 2)', 'drive(2, 1, 2, 0)', 'drive(3, 1, 0, 1)', 'arrive(4, 1, 1)'], but got ['drive(0, 0, 1, 0)', 'arrive(2, 0, 2)', 'drive(1, 1, 2, 0)', 'arrive(3, 1, 1)']